In [1]:
import imgTransformations
import cv2
import matplotlib.pyplot as plt
import os
from ipywidgets import FloatSlider, IntSlider, Output, VBox, Tab, interactive, Button, Label, SelectionSlider, Dropdown
import ipywidgets as widgets
import textwrap
import utils
import numpy as np
from IPython.display import clear_output

In [ ]:
dirname = os.path.abspath('')
# folder = os.path.join(dirname, 'MEFDatabase/source image sequences/Farmhouse_hdr-project.com/')
folder = os.path.join(dirname, 'MEFDatabase/source image sequences/')
entries = os.listdir(folder)
subfolders = [entry for entry in entries if os.path.isdir(os.path.join(folder, entry))]
images = []

def interactive_image_select(subfolder):
    global images
    imgSet = os.path.join(folder, subfolder)
    images = utils.load_images_from_folder(os.path.join(dirname, imgSet))
    images_per_row = 3

    num_inputs = len(images)

    # Calculate the number of rows needed for inputs and results
    input_rows = (num_inputs + images_per_row - 1) // images_per_row

    # Create the figure and axes
    fig, axes = plt.subplots(input_rows, images_per_row, figsize=(20,20))
    axes = axes.flatten()  # Flatten to easily iterate over all axes
    # Plot input images
    for i, img in enumerate(images):
        ax = axes[i]
        ax.imshow(img)
        ax.set_title(f'Input {i + 1}')
        ax.axis('off')

selectionWidget = interactive(interactive_image_select,
                              subfolder = Dropdown(
                                  options = subfolders,
                                  value = 'Farmhouse_hdr-project.com',
                                  description = 'Image set',
                                  disabled = False
                              )
                              )
display(selectionWidget)

# images = utils.load_images_from_folder(folder)


In [ ]:

import transformations._transformations

def interactive_average_fusion():
    title, fused = imgTransformations.average_fusion(images)
    plt.figure(figsize=(8,8))
    plt.imshow(fused)
    plt.title(title)
    plt.axis('off')
    plt.show()

def interactive_mertens_fusion():
    title, fused = imgTransformations.mertens_fusion(images)
    plt.figure(figsize=(8,8))
    plt.imshow(fused)
    plt.title(title)
    plt.axis('off')
    plt.show()

def interactive_exposure_fusion():
    title, fused = imgTransformations.exposure_fusion(images)
    plt.figure(figsize=(8,8))
    plt.imshow(fused)
    plt.title(title)
    plt.axis('off')
    plt.show()

def interactive_exposure_compensation_fusion():
    title, fused = imgTransformations.exposure_compensation_fusion(images)
    plt.figure(figsize=(8,8))
    plt.imshow(fused)
    plt.title(title)
    plt.axis('off')
    plt.show()

# Functions with numeric parameters
def interactive_laplacian_pyramid_fusion(levels):
    title, fused = imgTransformations.laplacian_pyramid_fusion(images, levels=levels)
    plt.figure(figsize=(8,8))
    plt.imshow(fused)
    plt.title(f"{title} (levels: {levels})")
    plt.axis('off')
    plt.show()

def interactive_enhanced_exposure_fusion(sigma, epsilon, kernel_dim):
    title, fused = imgTransformations.enhanced_exposure_fusion(images, sigma=sigma, epsilon=epsilon, blur_kernel=(kernel_dim,kernel_dim))
    plt.figure(figsize=(8,8))
    plt.imshow(fused)
    plt.title(f"{title} (sigma: {sigma:.2f}, epsilon: {epsilon})")
    plt.axis('off')
    plt.show()

def interactive_domain_transform_fusion(sigmaSpatial, sigmaColor, epsilon):
    title, fused = imgTransformations.domain_transform_fusion(images, sigmaSpatial=sigmaSpatial, sigmaColor=sigmaColor, epsilon=epsilon, homebrew_dt=False)
    plt.figure(figsize=(8,8))
    plt.imshow(fused)
    plt.title(f"{title} (sigmaSpatial: {sigmaSpatial}, sigmaColor: {sigmaColor}, epsilon: {epsilon})")
    plt.axis('off')
    plt.show()

def interactive_wavelet_fusion(level):
    title, fused = imgTransformations.wavelet_fusion(images, wavelet='db1', level=level)
    plt.figure(figsize=(8,8))
    plt.imshow(fused)
    plt.title(f"{title} (level: {level})")
    plt.axis('off')
    plt.show()

def interactive_dt_filter(sigmaSpatial, sigmaColor, num_iterations):
    guidance = cv2.cvtColor(images[0], cv2.COLOR_BGR2RGB).astype(np.float32)/255.0
    src = cv2.cvtColor(images[0], cv2.COLOR_BGR2GRAY).astype(np.float32)/255.0
    filtered = cv2.dt_filter(guidance, src, sigmaSpatial=sigmaSpatial, sigmaColor=sigmaColor, num_iterations=num_iterations)
    plt.figure(figsize=(8,8))
    plt.imshow(filtered, cmap='gray')
    plt.title(f"dt_filter (sigmaSpatial: {sigmaSpatial}, sigmaColor: {sigmaColor}, iterations: {num_iterations})")
    plt.axis('off')
    plt.show()

# --------------------------
# Create interactive widgets
# --------------------------

epsilon_values = np.round(np.logspace(-7, -5, num=10), decimals=10)

# Ensure the default value is in the list
default_epsilon = 1e-6
if default_epsilon not in epsilon_values:
    epsilon_values = np.append(epsilon_values, default_epsilon)
    epsilon_values = np.sort(epsilon_values)  # Keep sorted order


laplacian_widget = interactive(interactive_laplacian_pyramid_fusion, levels=IntSlider(min=1, max=10, step=1, value=4, description='Levels:'))
enhanced_widget = interactive(interactive_enhanced_exposure_fusion, 
                            sigma=FloatSlider(min=0.1, max=1.0, step=0.05, value=0.2, description='Sigma:'), 
                            epsilon=SelectionSlider(
                                options=[(f"{e:.1e}", e) for e in epsilon_values],  # Display as scientific notation
                                value=default_epsilon,
                                description="Epsilon:"
                            ),
                            kernel_dim = IntSlider(min=1, max=9, step=2, value=3, description="Kernel dimension:"))
domain_widget = interactive(interactive_domain_transform_fusion, 
                            sigmaSpatial=FloatSlider(min=10, max=100, step=5, value=60, description='sigmaSpatial:'), 
                            sigmaColor=FloatSlider(min=0.1, max=1.0, step=0.05, value=0.4, description='sigmaColor:'), 
                            epsilon=SelectionSlider(
                                options=[(f"{e:.1e}", e) for e in epsilon_values],  # Display as scientific notation
                                value=default_epsilon,
                                description="Epsilon:"
                            ))
wavelet_widget = interactive(interactive_wavelet_fusion, level=IntSlider(min=1, max=5, step=1, value=2, description='Level:'))

# For functions without adjustable numeric parameters, use buttons.
avg_button = Button(description="Run Average Fusion")
mertens_button = Button(description="Run Mertens Fusion")
exposure_button = Button(description="Run Exposure Fusion")
exp_comp_button = Button(description="Run Exposure Compensation Fusion")

out_avg = Output()
out_mertens = Output()
out_exposure = Output()
out_exp_comp = Output()

def run_avg(b):
    with out_avg:
        out_avg.clear_output(wait=True)
        interactive_average_fusion()
def run_mertens(b):
    with out_mertens:
        out_mertens.clear_output(wait=True)
        interactive_mertens_fusion()
def run_exposure(b):
    with out_exposure:
        out_exposure.clear_output(wait=True)
        interactive_exposure_fusion()
def run_exp_comp(b):
    with out_exp_comp:
        out_exp_comp.clear_output(wait=True)
        interactive_exposure_compensation_fusion()

avg_button.on_click(run_avg)
mertens_button.on_click(run_mertens)
exposure_button.on_click(run_exposure)
exp_comp_button.on_click(run_exp_comp)

# --------------------------
# Layout the widgets in a Tabbed interface
# --------------------------
tab = Tab(children=[
    VBox([Label("Average Fusion (no parameters):"), avg_button, out_avg]),
    VBox([Label("Mertens Fusion (no parameters):"), mertens_button, out_mertens]),
    VBox([Label("Laplacian Pyramid Fusion:"), laplacian_widget]),
    VBox([Label("Exposure Fusion (no parameters):"), exposure_button, out_exposure]),
    VBox([Label("Exposure Compensation Fusion (no parameters):"), exp_comp_button, out_exp_comp]),
    VBox([Label("Enhanced Exposure Fusion:"), enhanced_widget]),
    VBox([Label("Domain Transform Fusion:"), domain_widget]),
    VBox([Label("Wavelet Fusion:"), wavelet_widget]),
])
tab.set_title(0, "Average")
tab.set_title(1, "Mertens")
tab.set_title(2, "Laplacian")
tab.set_title(3, "Exposure")
tab.set_title(4, "Exp. Comp.")
tab.set_title(5, "Enhanced")
tab.set_title(6, "Domain Trans.")
tab.set_title(7, "Wavelet")

display(tab)

In [ ]:
import cv2

out = Output()

def show_weight_maps(image_index):
    """Function to compute and display weight maps for a given image index."""
    img = images[image_index]
    epsilon = 1e-6

    guidance = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

    gray = cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
    contrast = np.abs(cv2.Laplacian(gray, cv2.CV_32F))

    saturation = np.std(img, axis=2)

    well_exposedness = np.exp(-0.5 * ((img - 0.5) / 0.2) ** 2)
    well_exposedness = np.prod(well_exposedness, axis=2)

    weight = (contrast + epsilon) * (saturation + epsilon) * (well_exposedness + epsilon)

    # Apply Homebrew Domain Transform filter
    smooth_weight_homebrew = imgTransformations.dt_filter_homebrew(guidance, weight, sigmaSpatial=60, sigmaColor=0.4, num_iterations=2)

    # Apply OpenCV's dtFilter
    smooth_weight_opencv = cv2.ximgproc.dtFilter(img, weight.astype(np.float32), sigmaSpatial=60, sigmaColor=0.4, mode=1)

    with out:
        out.clear_output(wait=True)
        plt.figure(figsize=(18, 5))

        # Original Weight Map
        plt.subplot(1, 3, 1)
        plt.imshow(weight, cmap='gray')
        plt.title(f"Original Weight Map (Image {image_index})")
        plt.axis('off')

        # Homebrew Domain Transform
        plt.subplot(1, 3, 2)
        plt.imshow(smooth_weight_homebrew, cmap='gray')
        plt.title(f"Smoothed Weight Map (Homebrew DT, Image {image_index})")
        plt.axis('off')

        # OpenCV dtFilter
        plt.subplot(1, 3, 3)
        plt.imshow(smooth_weight_opencv, cmap='gray')
        plt.title(f"Smoothed Weight Map (OpenCV dtFilter, Image {image_index})")
        plt.axis('off')

        plt.tight_layout()
        plt.show()

# Create buttons dynamically
buttons = []
for i in range(len(images)):
    btn = Button(description=f"Show Image {i}")
    btn.on_click(lambda b, idx=i: show_weight_maps(idx))  # Capture correct index
    buttons.append(btn)

# Display buttons and output widget
display(VBox(buttons + [out]))

In [ ]:
#comparison between homebrew and opencv dt filter
title, fused = imgTransformations.domain_transform_fusion(images, homebrew_dt=False)
title_hb, fused_hb = imgTransformations.domain_transform_fusion(images, homebrew_dt=True)

plt.figure(figsize=(18, 5))
plt.subplot(1, 2, 1)
plt.imshow(fused)
plt.title(title)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(fused_hb)
plt.title(title_hb)
plt.axis('off')

In [ ]:
def calculate_entropy(image):
    """Calculates entropy for an RGB image by averaging entropy across channels."""
    entropy_values = []
    for channel in range(image.shape[-1]):  # Loop through R, G, B channels
        hist, _ = np.histogram(image[:, :, channel].ravel(), bins=256, range=(0, 256))
        hist = hist.astype(np.float32) / hist.sum()
        hist = hist[hist > 0]  # Remove zero probabilities
        entropy_values.append(-np.sum(hist * np.log2(hist)))
    return np.mean(entropy_values)  # Average entropy across channels

def spatial_frequency(image):
    """Calculates spatial frequency for an RGB image."""
    sf_values = []
    for channel in range(image.shape[-1]):  # Compute SF for each color channel
        rows, cols = image.shape[:2]
        row_freq = np.sqrt(np.sum(np.diff(image[:, :, channel], axis=0) ** 2) / (rows * cols))
        col_freq = np.sqrt(np.sum(np.diff(image[:, :, channel], axis=1) ** 2) / (rows * cols))
        sf_values.append(np.sqrt(row_freq**2 + col_freq**2))
    return np.mean(sf_values)  # Average SF across channels



In [ ]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import pandas as pd

# Define available fusion methods with their display names
fusion_methods = [
    ('Average Fusion', imgTransformations.average_fusion),
    ('Mertens Fusion', imgTransformations.mertens_fusion),
    ('Laplacian Pyramid', imgTransformations.laplacian_pyramid_fusion),
    ('Exposure Fusion', imgTransformations.exposure_fusion),
    ('Exposure Compensation', imgTransformations.exposure_compensation_fusion),
    ('Enhanced Exposure', imgTransformations.enhanced_exposure_fusion),
    ('Domain Transform', imgTransformations.domain_transform_fusion),
    ('Domain Transform - Homebrew', imgTransformations.domain_transform_fusion),
    ('Wavelet Fusion', imgTransformations.wavelet_fusion)
]

# Create checkboxes for method selection
method_checkboxes = [widgets.Checkbox(description=name, value=False) for name, _ in fusion_methods]
checkboxes_box = widgets.VBox([widgets.Label("Select Fusion Methods:")] + method_checkboxes)

# Create compare button
compare_button = widgets.Button(description="Compare Selected Methods")
output = widgets.Output()

def calculate_metrics(img1, img2):
    """Calculate similarity metrics between two images"""
    if img1.shape != img2.shape:
        img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))
    
    # Convert to grayscale for SSIM/PSNR
    gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    
    # Calculate SSIM
    ssim_value = ssim(gray1, gray2, data_range=255)
    
    # Calculate PSNR with error handling
    mse = np.mean((gray1 - gray2) ** 2)
    if mse == 0:
        psnr_value = np.inf
    else:
        with np.errstate(divide='ignore'):
            psnr_value = 20 * np.log10(255) - 10 * np.log10(mse)
    
    return ssim_value, psnr_value

def on_compare_button_clicked(b):
    with output:
        clear_output(wait=True)
        selected_methods = [(name, func) for (name, func), cb in zip(fusion_methods, method_checkboxes) if cb.value]
        
        if len(selected_methods) < 2:
            print("Please select at least 2 methods!")
            return
        
        # Generate fused images
        results = []
        for name, func in selected_methods:
            try:
                # Handle methods with required parameters
                if func == imgTransformations.laplacian_pyramid_fusion:
                    title, fused = func(images, levels=4)
                elif func == imgTransformations.enhanced_exposure_fusion:
                    title, fused = func(images, sigma=0.2, epsilon=1e-6)
                elif name == 'Domain Transform - Homebrew':
                    title, fused = func(images, sigmaSpatial=60, sigmaColor=0.4, homebrew_dt=True)
                elif func == imgTransformations.domain_transform_fusion:
                    title, fused = func(images, sigmaSpatial=60, sigmaColor=0.4)
                
                else:
                    title, fused = func(images)
                results.append((title, fused))
            except Exception as e:
                print(f"Error in {name}: {str(e)}")
                return
        
        # Display fused images
        fig, axes = plt.subplots(1, len(results), figsize=(20, 5))
        if len(results) == 1:
            axes = [axes]
            
        for ax, (title, img) in zip(axes, results):
            ax.imshow(img)
            ax.set_title(title)
            ax.axis('off')
        plt.show()
        
        # Calculate metrics
        num_images = len(results)
        metric_table = []
        similarity_matrix = np.zeros((num_images, num_images, 2))
        
        # Calculate individual metrics
        individual_metrics = []
        for title, img in results:
            entropy = calculate_entropy(img)
            sf = spatial_frequency(img)
            individual_metrics.append((entropy, sf))
        
        # Calculate pairwise metrics
        for i in range(num_images):
            for j in range(i, num_images):
                ssim_val, psnr_val = calculate_metrics(results[i][1], results[j][1])
                similarity_matrix[i, j] = (ssim_val, psnr_val)
                similarity_matrix[j, i] = (ssim_val, psnr_val)
        
        # Display metrics
        df_individual = pd.DataFrame([(title, f"{entropy:.4f}", f"{sf:.4f}") 
                                    for (title, _), (entropy, sf) in zip(results, individual_metrics)],
                                columns=['Method', 'Entropy', 'Spatial Frequency'])
        
        # Style individual metrics table
        individual_styler = (df_individual.style
                            .set_caption("Individual Image Metrics")
                            .set_properties(**{'text-align': 'left', 
                                            'font-size': '14px'})
                            .hide(axis='index')
                            .format(precision=4))
        display(individual_styler)

        # Create similarity matrices with NaN handling
        methods = [title for title, _ in results]
        
        ssim_matrix = np.zeros((len(methods), len(methods)))
        psnr_matrix = np.zeros((len(methods), len(methods)))
        for i in range(len(methods)):
            for j in range(len(methods)):
                ssim_val, psnr_val = similarity_matrix[i, j]
                ssim_matrix[i, j] = ssim_val
                psnr_matrix[i, j] = psnr_val if not np.isinf(psnr_val) else np.nan

        # Create formatted DataFrames
        ssim_df = pd.DataFrame(ssim_matrix, index=methods, columns=methods)
        psnr_df = pd.DataFrame(psnr_matrix, index=methods, columns=methods)

        # Formatting functions with special handling
        def format_ssim(val):
            return f"{val:.3f}" if not np.isnan(val) else ""
            
        def format_psnr(val):
            if np.isinf(val):
                return "∞"
            if np.isnan(val):
                return "N/A"
            return f"{val:.2f}"

        # Create styled tables with titles
        print("\n")
        ssim_styler = (ssim_df.style
                    .set_caption("Structural Similarity Index (SSIM)")
                    .format(format_ssim)
                    .background_gradient(cmap='Blues', axis=None, 
                                        vmin=0, vmax=1)
                    .set_properties(**{'font-size': '12px'}))

        # Update the PSNR styler creation
        finite_psnr = psnr_df.replace([np.inf, -np.inf], np.nan)
        vmin = finite_psnr.min().min()
        vmax = finite_psnr.max().max()

        psnr_styler = (psnr_df.style
                    .set_caption("Peak Signal-to-Noise Ratio (PSNR)")
                    .format(format_psnr)
                    .background_gradient(cmap='Greens', axis=None,
                                        vmin=vmin, vmax=vmax,
                                        subset=pd.IndexSlice[:, :])
                    .set_properties(**{'font-size': '12px'}))
        
        display(ssim_styler)
        print("\n")
        display(psnr_styler)

compare_button.on_click(on_compare_button_clicked)

# Display the widgets
display(widgets.VBox([checkboxes_box, compare_button, output]))